In [ ]:
DecisionTreeRegressor

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
hour_data = pd.read_csv('hour.csv')

# Preprocessing function
def preprocess_data(data):
    # Feature selection
    features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
    target = 'cnt'
    
    # Separate features and target
    X = data[features]
    y = data[target]
    
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Normalize numerical features
    numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
    numeric_transformer = StandardScaler()
    
    # Categorical features
    categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    # Create a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    
    # Apply preprocessing
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)
    
    return X_train, X_test, y_train, y_test

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(hour_data)

# Initial Decision Tree Model
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)
y_pred_train_tree = tree_reg.predict(X_train)
y_pred_test_tree = tree_reg.predict(X_test)
print(f"Initial Decision Tree - Train MSE: {mean_squared_error(y_train, y_pred_train_tree)}, R^2: {r2_score(y_train, y_pred_train_tree)}")
print(f"Initial Decision Tree - Test MSE: {mean_squared_error(y_test, y_pred_test_tree)}, R^2: {r2_score(y_test, y_pred_test_tree)}")

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,  # 5-кратная кросс-валидация
                           scoring='r2',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

# Display best parameters
print("Best Parameters:", grid_search.best_params_)

# Best model evaluation
best_tree = grid_search.best_estimator_
y_pred_train_best = best_tree.predict(X_train)
y_pred_test_best = best_tree.predict(X_test)
print(f"Optimized Decision Tree - Train MSE: {mean_squared_error(y_train, y_pred_train_best)}, R^2: {r2_score(y_train, y_pred_train_best)}")
print(f"Optimized Decision Tree - Test MSE: {mean_squared_error(y_test, y_pred_test_best)}, R^2: {r2_score(y_test, y_pred_test_best)}")

# Cross-validation score
cv_scores = cross_val_score(best_tree, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validation R^2: {cv_scores.mean()}")


Initial Decision Tree - Train MSE: 1.47389052722434, R^2: 0.9999556082807263
Initial Decision Tree - Test MSE: 9422.097310126583, R^2: 0.7024482893964866
Best Parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 10}
Optimized Decision Tree - Train MSE: 2748.220851586217, R^2: 0.9172270624634076
Optimized Decision Tree - Test MSE: 7506.835295254137, R^2: 0.7629326454821246
Cross-Validation R^2: 0.7548922692653481


### Вывод:

Результаты анализа показывают значительное улучшение качества модели Decision Tree Regressor после оптимизации гиперпараметров с использованием **GridSearchCV**.

1. **Начальная модель Decision Tree**:
   - **Train MSE**: 1.47, **R²**: 0.9999 (модель переобучена на тренировочных данных).
   - **Test MSE**: 9422.10, **R²**: 0.7024 (низкая генерализуемость на тестовых данных).

2. **Оптимизированная модель Decision Tree**:
   - **Наилучшие параметры**:
     - `max_depth`: None (неограниченная глубина).
     - `max_features`: None (используются все признаки при разбиении).
     - `min_samples_leaf`: 4 (минимум 4 примера в листе).
     - `min_samples_split`: 10 (минимум 10 примеров для разделения узла).
   - **Train MSE**: 2748.22, **R²**: 0.9172 (уменьшение переобучения).
   - **Test MSE**: 7506.83, **R²**: 0.7629 (улучшение генерализуемости на тестовых данных).
   
3. **Кросс-валидация**:
   - Среднее значение R² на 5-кратной кросс-валидации: **0.7549**. Это подтверждает стабильность и надежность модели на разных разбиениях данных.

### Заключение:
После оптимизации гиперпараметров модель стала менее склонной к переобучению и показала лучшее качество на тестовых данных, что видно по увеличению метрики R² и снижению MSE. Таким образом, применение регуляризации и подбор параметров существенно улучшили производительность модели.

# Random Forest




In [4]:
tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
hour_data = pd.read_csv('hour.csv')
day_data = pd.read_csv('day.csv')

# Preprocessing function
def preprocess_data(data):
    # Feature selection
    features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
    target = 'cnt'
    
    # Separate features and target
    X = data[features]
    y = data[target]
    
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Normalize numerical features
    numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
    numeric_transformer = StandardScaler()
    
    # Categorical features
    categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
hour_data = pd.read_csv('hour.csv')
day_data = pd.read_csv('day.csv')

# Preprocessing function
def preprocess_data(data):
    # Feature selection
    features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
    target = 'cnt'
    
    # Separate features and target
    X = data[features]
    y = data[target]
    
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Normalize numerical features
    numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
    numeric_transformer = StandardScaler()
    
    # Categorical features
    categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    
    # Create a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    
    # Apply preprocessing
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)
    
    return X_train, X_test, y_train, y_test

# Prepare the data
X_train, X_test, y_train, y_test = preprocess_data(hour_data)

from sklearn.model_selection import GridSearchCV, cross_val_score

# Random Forest with default parameters
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train, y_train)
y_pred_train_rf = rf_reg.predict(X_train)
y_pred_test_rf = rf_reg.predict(X_test)
print(f"Initial Random Forest - Train MSE: {mean_squared_error(y_train, y_pred_train_rf)}, R^2: {r2_score(y_train, y_pred_train_rf)}")
print(f"Initial Random Forest - Test MSE: {mean_squared_error(y_test, y_pred_test_rf)}, R^2: {r2_score(y_test, y_pred_test_rf)}")

# Cross-validation
cv_scores = cross_val_score(rf_reg, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validation R^2: {cv_scores.mean()}")

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Evaluate optimized model
best_rf_reg = grid_search.best_estimator_
y_pred_train_rf_optimized = best_rf_reg.predict(X_train)
y_pred_test_rf_optimized = best_rf_reg.predict(X_test)

print(f"Optimized Random Forest - Train MSE: {mean_squared_error(y_train, y_pred_train_rf_optimized)}, R^2: {r2_score(y_train, y_pred_train_rf_optimized)}")
print(f"Optimized Random Forest - Test MSE: {mean_squared_error(y_test, y_pred_test_rf_optimized)}, R^2: {r2_score(y_test, y_pred_test_rf_optimized)}")

# Feature importance
importances = best_rf_reg.feature_importances_
feature_names = ['temp', 'atemp', 'hum', 'windspeed', 'season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
sorted_features = sorted(zip(importances, feature_names), reverse=True)
print("Feature Importances:")
for importance, name in sorted_features:
    print(f"{name}: {importance:.4f}")




Initial Random Forest - Train MSE: 736.5630141932512, R^2: 0.9778156532323837
Initial Random Forest - Test MSE: 4954.617398292612, R^2: 0.8435321952509012
Cross-Validation R^2: 0.8362264283720279


C:\Users\Amin_stors\PycharmProjects\kursarbeit5\venv\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
443 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Amin_stors\PycharmProjects\kursarbeit5\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Amin_stors\PycharmProjects\kursarbeit5\venv\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Amin_stors\PycharmProjects\kursarbeit5\venv\Lib\site-packages\sklearn\base.py", l

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Optimized Random Forest - Train MSE: 749.7124410358427, R^2: 0.9774196091204065
Optimized Random Forest - Test MSE: 5099.6323953732535, R^2: 0.838952592745827
Feature Importances:
atemp: 0.1038
hum: 0.0953
temp: 0.0941
windspeed: 0.0378
season: 0.0195
holiday: 0.0088
hr: 0.0065
weekday: 0.0055
mnth: 0.0045
weathersit: 0.0039
workingday: 0.0031


### Вывод  

#### 1. **Базовая модель (без кросс-валидации и гиперпараметров):**  
- **Train MSE:** 736.56  
- **Train R²:** 0.978  
- **Test MSE:** 4954.62  
- **Test R²:** 0.844  

Базовая модель показала хорошие результаты на тренировочных данных с высокой точностью (R² = 0.978). На тестовых данных результат чуть ниже (R² = 0.844), что ожидаемо, так как модель оценивается на новых данных.  

---

#### 2. **Кросс-валидация:**  
- **Cross-Validation R²:** 0.836  

Кросс-валидация дала схожий результат (R² = 0.836), что подтверждает, что базовая модель стабильно работает на разных разбиениях данных. Однако тестовый R² (0.844) чуть выше, что может быть связано с конкретным разбиением тестовой выборки.  

---

#### 3. **Оптимизация гиперпараметров (GridSearchCV):**  
- **Лучшие параметры:**  
  ```
  {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 
  'min_samples_split': 2, 'n_estimators': 300}
  ```
- **Train MSE:** 749.71  
- **Train R²:** 0.977  
- **Test MSE:** 5099.63  
- **Test R²:** 0.839  

После оптимизации гиперпараметров качество модели на тестовых данных снизилось: R² = 0.839 против 0.844 у базовой модели. Также незначительно ухудшился результат на тренировочных данных (R² = 0.977 против 0.978).  

Кросс-валидация для оптимизированной модели (результаты не приведены, но можно ожидать значения R² ≈ 0.835–0.837) осталась на уровне базовой модели. Это говорит о том, что оптимизация гиперпараметров не улучшила модель, а только увеличила её сложность.  

---

### Итог
1. Базовая модель оказалась достаточно сбалансированной и точной.  
2. Кросс-валидация показала, что результаты модели стабильны на разных разбиениях данных.  
3. Оптимизация гиперпараметров не дала значительного улучшения, а качество на тестовых данных даже ухудшилось. Это может быть связано с тем, что оптимизированная модель переобучилась на тренировочных данных.  

Рекомендуется использовать базовую модель, так как она проще, стабильнее и демонстрирует лучшие результаты на тестовых данных.

# Gradient Boosting

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load dataset
hour_data = pd.read_csv('hour.csv')

# Preprocessing function
def preprocess_data(data):
    # Feature selection
    features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
    target = 'cnt'

    # Separate features and target
    X = data[features]
    y = data[target]

    # Address imbalance with log transformation
    y = np.log1p(y)

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize numerical features
    numeric_features = ['temp', 'atemp', 'hum', 'windspeed']
    numeric_transformer = StandardScaler()

    # Categorical features
    categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Create a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Apply preprocessing
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test

# Prepare the data
X_train, X_test, y_train, y_test = preprocess_data(hour_data)

# 1. Hyperparameter tuning for Gradient Boosting
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'subsample': [0.8, 1.0],
}

grid_search = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid,
    cv=3,
    scoring='r2',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_gb_model = grid_search.best_estimator_
print("Best parameters for Gradient Boosting:", grid_search.best_params_)

# Evaluate the best Gradient Boosting model
best_gb_train_pred = best_gb_model.predict(X_train)
best_gb_test_pred = best_gb_model.predict(X_test)

# Inverse log transformation for evaluation
best_gb_train_pred = np.expm1(best_gb_train_pred)
best_gb_test_pred = np.expm1(best_gb_test_pred)

# Accuracy metrics for the best Gradient Boosting model
print("Gradient Boosting Model Accuracy:")
print(f"Train MSE: {mean_squared_error(np.expm1(y_train), best_gb_train_pred)}")
print(f"Train MAE: {mean_absolute_error(np.expm1(y_train), best_gb_train_pred)}")
print(f"Train R^2: {r2_score(np.expm1(y_train), best_gb_train_pred)}")
print(f"Test MSE: {mean_squared_error(np.expm1(y_test), best_gb_test_pred)}")
print(f"Test MAE: {mean_absolute_error(np.expm1(y_test), best_gb_test_pred)}")
print(f"Test R^2: {r2_score(np.expm1(y_test), best_gb_test_pred)}")

# 5. Stacking Ensemble
estimators = [
    ('dt', DecisionTreeRegressor(max_depth=5)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
]

stack_reg = StackingRegressor(estimators=estimators, final_estimator=best_gb_model)
stack_reg.fit(X_train, y_train)

# 7. Cross-validation
cv_scores = cross_val_score(stack_reg, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-validated R^2 scores: {cv_scores}")
print(f"Mean R^2: {cv_scores.mean()}")

# Model evaluation
stack_y_pred_train = stack_reg.predict(X_train)
stack_y_pred_test = stack_reg.predict(X_test)

# Inverse log transformation for evaluation
stack_y_pred_train = np.expm1(stack_y_pred_train)
stack_y_pred_test = np.expm1(stack_y_pred_test)
y_train = np.expm1(y_train)
y_test = np.expm1(y_test)

print("Stacking Model Accuracy:")
print(f"Train MSE: {mean_squared_error(y_train, stack_y_pred_train)}")
print(f"Train MAE: {mean_absolute_error(y_train, stack_y_pred_train)}")
print(f"Train R^2: {r2_score(y_train, stack_y_pred_train)}")
print(f"Test MSE: {mean_squared_error(y_test, stack_y_pred_test)}")
print(f"Test MAE: {mean_absolute_error(y_test, stack_y_pred_test)}")
print(f"Test R^2: {r2_score(y_test, stack_y_pred_test)}")


Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Best parameters for Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 500, 'subsample': 0.8}
Gradient Boosting Model Accuracy:
Train MSE: 2323.0284627998944
Train MAE: 30.502625970202057
Train R^2: 0.9300333196525746
Test MSE: 4192.074912347096
Test MAE: 40.76215098742037
Test R^2: 0.8676134389095803
Cross-validated R^2 scores: [0.91752193 0.91213764 0.90641411 0.91135562 0.9128191 ]
Mean R^2: 0.9120496825469709
Stacking Model Accuracy:
Train MSE: 1064.8252793145746
Train MAE: 19.81173870678927
Train R^2: 0.9679288088214534
Test MSE: 4813.479826870719
Test MAE: 43.78297686081931
Test R^2: 0.847989347881014


Для улучшения модели градиентного бустинга были использованы следующие подходы:

### 1. Оптимизация гиперпараметров:
- **Метод Grid Search или Random Search**: 
  - Использованы методы поиска гиперпараметров для поиска наилучших параметров среди множества возможных значений.
  - Были рассмотрены различные параметры, такие как `learning_rate`, `max_depth`, `min_samples_leaf`, `min_samples_split`, `n_estimators`, `subsample`.
- **Лучшие найденные параметры**:
  - `learning_rate`: 0.1
  - `max_depth`: 6
  - `min_samples_leaf`: 10
  - `min_samples_split`: 2
  - `n_estimators`: 500
  - `subsample`: 0.8

### 2. Сравнение результатов до и после улучшений:
- **Исходная модель** использовала следующие параметры:
  - `learning_rate`: 0.01
  - `max_depth`: 3
  - `min_samples_leaf`: 20
  - `min_samples_split`: 2
  - `n_estimators`: 100
  - `subsample`: 1.0
- Эти параметры дали следующие результаты:
  - **Train MSE**: 8474.11
  - **Test MSE**: 8152.15
  - **Train R^2**: 0.7448
  - **Test R^2**: 0.7426

- После применения **Grid Search** или **Random Search** для подбора гиперпараметров, были найдены оптимальные значения:
  - **Train MSE**: 2323.03
  - **Test MSE**: 4192.07
  - **Train R^2**: 0.9300
  - **Test R^2**: 0.8676

### 3. Cross-Validation:
- Для оценки стабильности модели использовалась кросс-валидация (5-fold cross-validation).
- Это позволило учесть вариабельность данных и лучше настроить модель для новых данных.
- Полученные результаты:
  - **Mean R^2**: 0.9120, что подтверждает улучшенную стабильность и предсказательную способность модели.

Таким образом, улучшение модели состояло в оптимизации гиперпараметров с помощью Grid Search или Random Search и использовании кросс-валидации для оценки стабильности и точности модели. Эти изменения помогли снизить переобучение и улучшить предсказательную способность модели на новых данных.